In [ ]:
# to test any error in the terminal, use:
# ipython3 -c "%run notebook.ipynb"

In [1]:
%%bash
# remove everything except this notebook
shopt -s extglob
rm -rf !("notebook.ipynb") 

In [2]:
!conda env list

# conda environments:
#
base                     /opt/conda



In [1]:
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from meeko import MoleculePreparation
from vina import Vina
import numpy as np
import vina

try:
    print("vina version:", vina.__version__)
except AttributeError:
    print("vina.__version__ not available:")

receptor_string = "ATOM      1  C   UNL     1       0.000   0.000   0.000  0.00  0.00    +0.000 C "
rec_fn = "my_rec_is_just_a_carbon.pdbqt"
print(receptor_string, file=open(rec_fn, "w"))

maps_center = [0, 0, 0]
maps_size_angstroms = [10, 10, 10]

pyridine = Chem.MolFromSmiles("C1=CC=CN=C1")
pyridine = Chem.AddHs(pyridine)
etkdg_params = rdDistGeom.ETKDGv3()
rdDistGeom.EmbedMolecule(pyridine, etkdg_params)

mkprep = MoleculePreparation()
mkprep.prepare(pyridine)
lig_string = mkprep.write_pdbqt_string()

v = Vina()
v.set_receptor(rec_fn)
v.set_ligand_from_string(lig_string)
v.compute_vina_maps(center=maps_center, box_size=maps_size_angstroms)

def print_ligand_center(molsetup):
    lig_xyz = []
    for atom_index, is_atom_ignored in molsetup.atom_ignore.items():
        if not is_atom_ignored:
            lig_xyz.append(molsetup.coord[atom_index].copy())
    lig_xyz = np.array(lig_xyz)
    print("ligand center: %8.3f %8.3f %8.3f" % tuple(np.mean(lig_xyz, 0)))

print_ligand_center(mkprep.setup)
score = v.score()
print("score:", score)

if False: # this stops the kernel without even throwing an exception
    # translate ligand out of box
    for atom_index in mkprep.setup.coord:
        x, y, z = mkprep.setup.coord[atom_index]
        mkprep.setup.coord[atom_index] = np.array([x + 200, y + 200, z + 200])

    print_ligand_center(mkprep.setup)
    lig_string = mkprep.write_pdbqt_string()
    v.set_ligand_from_string(lig_string)
    score = v.score()
    print("score:", score)

vina version: 1.2.3
Computing Vina grid ... ligand center:   -0.011   -0.218    0.005
score: [29.066 29.066  0.     0.     0.     0.     0.     0.   ]
done.
